In [1]:
import os
import chromadb
from chromadb.config import Settings
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer


load_dotenv()

client = chromadb.HttpClient(
    host="localhost",
    port=6854,
    settings=Settings(
        chroma_client_auth_provider="chromadb.auth.basic_authn.BasicAuthClientProvider",
        chroma_client_auth_credentials=os.getenv("CHROMA_CLIENT_AUTH_CREDENTIALS")
    )
)

client.heartbeat()


/home/halil/.pyenv/versions/DI502/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


1730827997743812809

In [2]:
from sentence_transformers import SentenceTransformer
import pandas as pd

file_path = '/home/halil/Desktop/MeETU/Data/FAQs_eng.xlsx'
excel_data = pd.read_excel(file_path)

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
excel_data['combined_text'] = excel_data['question'] + " " + excel_data['answer']
excel_data['embeddings'] = excel_data['combined_text'].apply(lambda x: embedding_model.encode(x))
excel_data[['link', 'question', 'embeddings','combined_text']].head()

,link,question,embeddings,combined_text
0,https://faq.cc.metu.edu.tr/faq/how-can-i-conne...,How can I connect to eduroam wireless network ...,"[-0.08665929, -0.019397035, -0.01732739, -0.01...",How can I connect to eduroam wireless network ...
1,https://faq.cc.metu.edu.tr/faq/meturoam,How can I connect to meturoam network?,"[-0.07703145, -0.07036505, 0.0060302727, -0.02...",How can I connect to meturoam network? \n\nA d...
2,https://faq.cc.metu.edu.tr/faq/guest-account-c...,A guest account created for me. How can I conn...,"[-0.07025432, 0.000523283, -0.022801565, -0.03...",A guest account created for me. How can I conn...
3,https://faq.cc.metu.edu.tr/faq/how-can-i-acces...,How can I access to The Resources of METU Libr...,"[-0.015727324, -0.06908474, -0.073817015, -0.0...",How can I access to The Resources of METU Libr...
4,https://faq.cc.metu.edu.tr/faq/how-managed-met...,How to managed METU Departments/Units Wired Ne...,"[0.018322488, -0.08207159, -0.005806347, 0.035...",How to managed METU Departments/Units Wired Ne...


In [6]:
collection = client.get_or_create_collection(name="faq_embeddings")
for index, row in excel_data.iterrows():
    collection.add(
        embeddings=[row['embeddings']],
        ids=[str(index)],
        documents=[row['combined_text']],  # Store the combined question-answer text or other content
        metadatas=[{
            "link": row['link'],
            "question": row['question']
        }]
    )

In [7]:
collection = client.get_collection(name="faq_embeddings")

In [67]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
test_question = "How can I connect the Wifi in METU"
query_embedding = embedding_model.encode(test_question)

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3
)

In [68]:
results["ids"]

[['1', '65', '62']]

In [79]:
print(collection.get(ids="65")["documents"][0])

metuconnect 

If you do not have a password for meturoam and do not have access to METU wireless network, then you can connect to metuconnect SSID in order to connect to meturoam. After connecting to metuconnect, please login to https://netregister.metu.edu.tr and follow the instructions to create a new meturoam password. Users who already have meturoam passwords do not need to connect to metuconnect.
There are access restrictions in the metuconnect network.




In [96]:
documents = []
for i in range(len(results['metadatas'][0])):
    #print(results['ids'][0][i] if 'ids' in results and results['ids'][0] else None)
    #print(results['metadatas'][0][i][0] if results['metadatas'][i] else {})
    print(results['metadatas'][0][i])

{'link': 'https://faq.cc.metu.edu.tr/faq/meturoam', 'question': 'How can I connect to meturoam network?'}
{'link': 'https://faq.cc.metu.edu.tr/faq/metuconnect', 'question': 'metuconnect'}
{'link': 'https://faq.cc.metu.edu.tr/faq/how-metu-wireless-network-managed', 'question': 'How is METU Wireless Network managed?'}


In [94]:
results['metadatas']

[[{'link': 'https://faq.cc.metu.edu.tr/faq/meturoam',
   'question': 'How can I connect to meturoam network?'},
  {'link': 'https://faq.cc.metu.edu.tr/faq/metuconnect',
   'question': 'metuconnect'},
  {'link': 'https://faq.cc.metu.edu.tr/faq/how-metu-wireless-network-managed',
   'question': 'How is METU Wireless Network managed?'}]]

In [83]:
len(results['metadatas'][0])

3

In [72]:
print(documents[0]["content"])

How can I connect to meturoam network? 

A device must support WPA2 Enterprise with PEAP/MSCHAPv2 in order to be able to connect to METU wireless network, meturoam. Please click here for a list of some popular devices that are known to NOT work on the meturoam wireless network.
To connect to the meturoam wireless network, you need a METU usercode and a password for meturoam. If you have a password, please continue to HOW-TO  section.
If you didn't create a password for meturoam or you want to renew your password, please visit https://netregister.metu.edu.tr and click Wireless icon, then click "meturoam" icon.

In the meturoam menu, please provide a new password to be used only for this service with your user code. (In some Android and IOS devices, some special characters like #, ', $ in the password may cause problems to connect to meturoam. If you can connect to meturoam via a computer but having problems with Android or IOS devices, please check your password for special characters a

In [100]:
class ChromaDBRetriever:
    def __init__(self, chroma_collection, embedding_model):
        self.collection = chroma_collection
        self.embedding_model = embedding_model

    def get_relevant_documents(self, query_text):
        query_embedding = self.embedding_model.encode(query_text)
        results = self.collection.query(
            query_embeddings=[query_embedding],
            n_results=1
        )
        documents = []
        for i in range(len(results['metadatas'][0])):
            doc_id = results['ids'][0][i] if 'ids' in results and results['ids'][0] else None
            metadata = results['metadatas'][0][i] if results['metadatas'][0][i] else {}
            document_content = results['documents'][0][i] if 'documents' in results and results['documents'][0] else 'No content available'
            
            documents.append({
                'id': doc_id,
                'content': document_content,
                'metadata': metadata
            })
            
        return documents

    
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
retriever = ChromaDBRetriever(collection, embedding_model)

query_results = retriever.get_relevant_documents("How can I connect the Wifi?")
if query_results and 'id' in query_results[0]:
    top_doc_id = query_results[0]['id']
else:
    top_doc_id = None


In [101]:
relevant_docs = retriever.get_relevant_documents("How can I connect the Wifi?")
context = " ".join([doc['content'] for doc in relevant_docs])

In [103]:
top_doc_id

'1'

In [102]:
print(context)

How can I connect to meturoam network? 

A device must support WPA2 Enterprise with PEAP/MSCHAPv2 in order to be able to connect to METU wireless network, meturoam. Please click here for a list of some popular devices that are known to NOT work on the meturoam wireless network.
To connect to the meturoam wireless network, you need a METU usercode and a password for meturoam. If you have a password, please continue to HOW-TO  section.
If you didn't create a password for meturoam or you want to renew your password, please visit https://netregister.metu.edu.tr and click Wireless icon, then click "meturoam" icon.

In the meturoam menu, please provide a new password to be used only for this service with your user code. (In some Android and IOS devices, some special characters like #, ', $ in the password may cause problems to connect to meturoam. If you can connect to meturoam via a computer but having problems with Android or IOS devices, please check your password for special characters a

In [24]:

results = collection.query(
    query_embeddings=[embedding_model.encode("How can I connect")],
    n_results=5
)

documents = []
for result_list in results['metadatas']:
    result = result_list[0]
    question = result.get('question', 'No question available')
    content = question + " " + result.get('link', '')
    documents.append({
        'content': content,
        'metadata': result
    })

In [25]:

documents = []
for result_list in results['metadatas']:
    result = result_list[0]
    question = result.get('question', 'No question available')
    content = question + " " + result.get('link', '')
    documents.append({
        'content': content,
        'metadata': result
    })

In [28]:
result.get('question', 'No question available')

'How can I connect to meturoam network?'

In [27]:
results['metadatas']

[[{'link': 'https://faq.cc.metu.edu.tr/faq/meturoam',
   'question': 'How can I connect to meturoam network?'},
  {'link': 'https://faq.cc.metu.edu.tr/faq/how-can-i-connect-eduroam-wireless-network-using-my-mac-computer',
   'question': 'How can I connect to eduroam wireless network using my Mac computer?'},
  {'link': 'https://faq.cc.metu.edu.tr/faq/it-seems-i-have-connection-wireless-network-i-cant-connect-internet-what-should-i-do',
   'question': "It seems I have a connection to the wireless network but I can't connect to the Internet. What should I do?"}]]

In [22]:
query_results

[{'content': 'How can I connect to meturoam network? https://faq.cc.metu.edu.tr/faq/meturoam',
  'metadata': {'link': 'https://faq.cc.metu.edu.tr/faq/meturoam',
   'question': 'How can I connect to meturoam network?'}}]

In [ ]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

test_question = "How can I connect the Wifi in METU"
query_embedding = embedding_model.encode(test_question)

query_results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3
)

for i, doc_id in enumerate(query_results['ids'][0]):
    question = query_results['metadatas'][0][i]['question']
    link = query_results['metadatas'][0][i]['link']
    answer = query_results['documents'][0][i]

    print(f"Document ID: {doc_id}")
    print(f"Question: {question}")
    print(f"Link: {link}")
    print(f"Answer: {answer}")
    print("\n---\n")


/home/halil/.pyenv/versions/DI502/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Document ID: 1
Question: How can I connect to meturoam network?
Link: https://faq.cc.metu.edu.tr/faq/meturoam
Answer: How can I connect to meturoam network? 

A device must support WPA2 Enterprise with PEAP/MSCHAPv2 in order to be able to connect to METU wireless network, meturoam. Please click here for a list of some popular devices that are known to NOT work on the meturoam wireless network.
To connect to the meturoam wireless network, you need a METU usercode and a password for meturoam. If you have a password, please continue to HOW-TO  section.
If you didn't create a password for meturoam or you want to renew your password, please visit https://netregister.metu.edu.tr and click Wireless icon, then click "meturoam" icon.

In the meturoam menu, please provide a new password to be used only for this service with your user code. (In some Android and IOS devices, some special characters like #, ', $ in the password may cause problems to connect to meturoam. If you can connect to meturo